In [7]:
import cv2
import os
import math
import json
import random

from data_preprocessing import DataPreprocessing
from feature_extraction import NASNetFeatureExtractor

class VideoProcessor:
    def __init__(self, input_json_file, output_folder, processing_steps=None):
        self.input_json_file = input_json_file
        self.output_folder = output_folder
        self.resize_width = 331
        self.resize_height = 331
        self.processing_steps = processing_steps
        self.dp = DataPreprocessing()
        self.fe = NASNetFeatureExtractor()
        self.output_dirs = {}

    def process_videos(self):
        try:
            self._create_output_directories()

            # Get the list of videos in the JSON file
            with open(self.input_json_file, 'r') as json_file:
                data = json.load(json_file)

            num_videos = len(data)
            num_steps = len(self.processing_steps)
            videos_per_step = math.ceil(num_videos / num_steps)

            # Shuffle the processing steps
            random.shuffle(self.processing_steps)

            # Process videos in the input folder
            for i, file in enumerate(data):
                video_path = file['SENTENCE_FILE_PATH']
                video_name = os.path.split(video_path)[-1].split('.')[0]

                # Determine the processing step for the video
                step_idx = i // videos_per_step
                step = self.processing_steps[step_idx]
                output_dir = self.output_dirs[step]

                # Create a subdirectory for the current video within the processing step directory
                video_dir = os.path.join(output_dir, video_name)
                # os.makedirs(video_dir, exist_ok=True)

                # Open the input video
                cap = cv2.VideoCapture(video_path)

                # Process frames in the video
                frame_count = 0
                output_json = []
                while cap.isOpened():
                    ret, frame = cap.read()
                    if not ret:
                        break

                    # Resize frame
                    resized_frame = self.dp.resize(frame, self.resize_width, self.resize_height)

                    # Perform the respective processing step on frames
                    if step == "crop":
                        cropped_frame = self.dp.crop(frame, 300, 0, 680, 720)
                        cropped_frame = self.dp.resize(cropped_frame, self.resize_width, self.resize_height)
                        # frame_path = os.path.join(video_dir, f"frame_{frame_count}.jpg")
                        # cv2.imwrite(frame_path, cropped_frame)
                        features = self.fe.extract_features_from_frame(cropped_frame)
                        frame_name = f"frame_{frame_count}"
                        frame_dict = {"frame_name": frame_name, "features": str(features)}
                        output_json.append(frame_dict)


                    elif step == "reduce_noise":
                        noise_reduced_frame = self.dp.reduce_noise(resized_frame, h=10, templateWindowSize=7,
                                                                   searchWindowSize=21)
                        # frame_path = os.path.join(video_dir, f"frame_{frame_count}.jpg")
                        # cv2.imwrite(frame_path, noise_reduced_frame)
                        features = self.fe.extract_features_from_frame(noise_reduced_frame)
                        frame_name = f"frame_{frame_count}"
                        frame_dict = {"frame_name": frame_name, "features": str(features)}
                        output_json.append(frame_dict)

                    elif step == "add_noise":
                        noise_added_frame = self.dp.add_noise(resized_frame, noise_type='gaussian', mean=0, std=1)
                        # frame_path = os.path.join(video_dir, f"frame_{frame_count}.jpg")
                        # cv2.imwrite(frame_path, noise_added_frame)
                        features = self.fe.extract_features_from_frame(noise_added_frame)
                        frame_name = f"frame_{frame_count}"
                        frame_dict = {"frame_name": frame_name, "features": str(features)}
                        output_json.append(frame_dict)

                    elif step == "rotate":
                        transformed_frame = self.dp.spatial_transformation(resized_frame, rotation_angle=90,
                                                                            flip_horizontal=False,
                                                                            flip_vertical=False)
                        # frame_path = os.path.join(video_dir, f"frame_{frame_count}.jpg")
                        # cv2.imwrite(frame_path, transformed_frame)
                        features = self.fe.extract_features_from_frame(transformed_frame)
                        frame_name = f"frame_{frame_count}"
                        frame_dict = {"frame_name": frame_name, "features": str(features)}
                        output_json.append(frame_dict)

                    elif step == "flip_horizontal":
                        transformed_frame = self.dp.spatial_transformation(resized_frame, rotation_angle=0,
                                                                            flip_horizontal=True,
                                                                            flip_vertical=False)
                        # frame_path = os.path.join(video_dir, f"frame_{frame_count}.jpg")
                        # cv2.imwrite(frame_path, transformed_frame)
                        features = self.fe.extract_features_from_frame(transformed_frame)
                        frame_name = f"frame_{frame_count}"
                        frame_dict = {"frame_name": frame_name, "features": str(features)}
                        output_json.append(frame_dict)

                    elif step == "flip_vertical":
                        transformed_frame = self.dp.spatial_transformation(resized_frame, rotation_angle=0,
                                                                            flip_horizontal=False,
                                                                            flip_vertical=True)
                        # frame_path = os.path.join(video_dir, f"frame_{frame_count}.jpg")
                        # cv2.imwrite(frame_path, transformed_frame)
                        features = self.fe.extract_features_from_frame(transformed_frame)
                        frame_name = f"frame_{frame_count}"
                        frame_dict = {"frame_name": frame_name, "features": str(features)}
                        output_json.append(frame_dict)

                    elif step == "brightness":
                        color_adjusted_frame = self.dp.color_jitter(resized_frame, brightness=0.5, contrast=1.0,
                                                                    saturation=1.0, hue=0.0)
                        # frame_path = os.path.join(video_dir, f"frame_{frame_count}.jpg")
                        # cv2.imwrite(frame_path, color_adjusted_frame)
                        features = self.fe.extract_features_from_frame(color_adjusted_frame)
                        frame_name = f"frame_{frame_count}"
                        frame_dict = {"frame_name": frame_name, "features": str(features)}
                        output_json.append(frame_dict)

                    elif step == "contrast":
                        color_adjusted_frame = self.dp.color_jitter(resized_frame, brightness=0.0, contrast=1.5,
                                                                    saturation=1.0, hue=0.0)
                        # frame_path = os.path.join(video_dir, f"frame_{frame_count}.jpg")
                        # cv2.imwrite(frame_path, color_adjusted_frame)
                        features = self.fe.extract_features_from_frame(color_adjusted_frame)
                        frame_name = f"frame_{frame_count}"
                        frame_dict = {"frame_name": frame_name, "features": str(features)}
                        output_json.append(frame_dict)

                    elif step == "saturation":
                        color_adjusted_frame = self.dp.color_jitter(resized_frame, brightness=0.0, contrast=1.0,
                                                                    saturation=1.5, hue=0.0)
                        # frame_path = os.path.join(video_dir, f"frame_{frame_count}.jpg")
                        # cv2.imwrite(frame_path, color_adjusted_frame)
                        features = self.fe.extract_features_from_frame(color_adjusted_frame)
                        frame_name = f"frame_{frame_count}"
                        frame_dict = {"frame_name": frame_name, "features": str(features)}
                        output_json.append(frame_dict)

                    elif step == "greyscale":
                        greyscale_frame = self.dp.grey_scale(resized_frame)
                        # frame_path = os.path.join(video_dir, f"frame_{frame_count}.jpg")
                        # cv2.imwrite(frame_path, greyscale_frame)
                        features = self.fe.extract_features_from_frame(greyscale_frame)
                        frame_name = f"frame_{frame_count}"
                        frame_dict = {"frame_name": frame_name, "features": str(features)}
                        output_json.append(frame_dict)

                    frame_count += 1

                # Release the video capture
                cap.release()
                # Open the JSON file in write mode
                with open(video_dir+'.json', 'w') as f:
                    # Write to the JSON file
                    json.dump(output_json, f, indent = 4)

            print("Video processing completed.")

        except Exception as e:
            print(f"Error occurred during video processing: {str(e)}")

    def _create_output_directories(self):
        try:
            for step in self.processing_steps:
                step_dir = os.path.join(self.output_folder, step)
                os.makedirs(step_dir, exist_ok=True)
                self.output_dirs[step] = step_dir
        except Exception as e:
            raise Exception(f"Error occurred while creating output directories: {str(e)}")


input_json_file = '../Dataset/test.json'
output_folder = '../Dataset/features/'

# processing_steps = ["crop"]
processing_steps = ["crop", "reduce_noise", "add_noise", "rotate", "brightness", "contrast", "saturation"]

processor = VideoProcessor(input_json_file, output_folder, processing_steps)
processor.process_videos()


1/1 [==============================] - 0s 341ms/step
